# Object Detection on Custom Data. 😎️

**#Just GPU Things**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Tue May  3 19:23:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Accessing Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**List of items in your "gdrive"**

**1) Clone the Darknet**



**Let's go Dark.**
As we will clone the darknet you may not be able to see it in your drive page, but you can explore it using linux commands(ie. cd / ls etc).

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15412, done.
remote: Total 15412 (delta 0), reused 0 (delta 0), pack-reused 15412
Receiving objects: 100% (15412/15412), 14.05 MiB | 18.52 MiB/s, done.
Resolving deltas: 100% (10354/10354), done.


**2) Now let's set the darknet with GPU.**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

We will copy the "yolov3-tiny.cfg" file from cfg directory to "yolov3_training.cfg".

In [ ]:
!cp cfg/yolov3-tiny.cfg cfg/yolov3_training.cfg

**Let's edit some configurations of the cfg file.**(pun)

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 24000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=12@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=12@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=12@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=51@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=51@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=51@' cfg/yolov3_training.cfg

**Let's create a folder named "yolov3_w1" to save the new weights.** (trash name, I know)

In [ ]:
#@title
# Create folder on google drive so that we can save there the weights
!mkdir "/content/gdrive/MyDrive/DLProject/yolov3_w1"

**We will make a file called "obj.data" in data directory that will keep all the classes names, here I have only used one class, so bear with me 😌️**

In [ ]:
!echo "bicycle\nboat\nbottle\nbus\ncar\ncat\nchair\ncup\dog\nmotorbike\npeople\ntable" > data/obj.names
!echo -e 'classes= 12\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /content/gdrive/MyDrive/yolov3_w1' > data/obj.data
# !mkdir data/obj

**For training we need a weight file to train and save it, let's use "darknet53.conv.74"**

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-05-03 19:26:40--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  58.4MB/s    in 2.7s    

2022-05-03 19:26:43 (58.4 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

For that you have to upload the data(including image and label file) on drive.
Now you have to edit the path in the given code.

In [ ]:
!unzip /content/gdrive/MyDrive/DLProject/dataUpdated.zip -d data/obj

Streaming output truncated to the last 5000 lines.
  inflating: data/obj/data/train/2015_01146.jpg  
  inflating: data/obj/data/train/2015_01147.jpg  
  inflating: data/obj/data/train/2015_01148.jpg  
  inflating: data/obj/data/train/2015_01149.jpg  
  inflating: data/obj/data/train/2015_01150.JPEG  
  inflating: data/obj/data/train/2015_01151.JPEG  
  inflating: data/obj/data/train/2015_01152.JPEG  
  inflating: data/obj/data/train/2015_01153.JPEG  
 extracting: data/obj/data/train/2015_01154.JPEG  
  inflating: data/obj/data/train/2015_01155.JPEG  
  inflating: data/obj/data/train/2015_01156.JPEG  
 extracting: data/obj/data/train/2015_01157.JPEG  
  inflating: data/obj/data/train/2015_01158.JPEG  
  inflating: data/obj/data/train/2015_01159.JPEG  
  inflating: data/obj/data/train/2015_01160.JPEG  
 extracting: data/obj/data/train/2015_01161.JPEG  
  inflating: data/obj/data/train/2015_01162.JPEG  
  inflating: data/obj/data/train/2015_01163.JPEG  
  inflating: data/obj/data/train/20

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re
import cv2

classes = {"0": "bicycle", "1" : "boat", "2" : "bottle" , "3" : "bus", "4":"car", "5":"cat" ,"6" : "chair" , "7":"cup","8":"dog","9":"motorbike","10":"people","11":"table"}
key_list = list(classes.keys())
val_list = list(classes.values())

txt_file_paths = glob.glob("data/obj/dataUpdated/train/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()
        print(file_path)
        if '% bbGt version=3\n' in lines:
          lines.remove('% bbGt version=3\n')
        text_converted = []
        for line in lines:
            j = line.split(' ')
            p = '.'.join(file_path.split('.')[:2])
            # print(file_path.split('.'))
            if os.path.exists(p):
              img = cv2.imread(p)
              if img is not None:
                size = img.shape
              x1, x2, y1, y2 = int(j[1]), int(j[2]), int(j[3]) + int(j[1]), int(j[4]) + int(j[2])
              xmax, xmin = max(x1,x2) , min(x1,x2)
              ymax, ymin = max(y1,y2), min(y1,y2)
              dw = 1./size[1]
              dh = 1./size[0]
              x = (xmin + xmax)/2.0
              y = (ymin + ymax)/2.0
              w = xmax - xmin
              h = ymax - ymin
              x = x*dw
              w = w*dw
              y = y*dh
              h = h*dh
              text = "{} {} {} {} {}".format(key_list[val_list.index(j[0].lower())], x,y,w,h)
              text_converted.append(text)
        # Write file
        os.remove(file_path)
        filename = file_path.split('/')[4].split('.')[0] + '.txt'
        file_path = '/'.join(file_path.split('/')[:4]) + '/' + filename
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

In [ ]:
import glob
images_list = glob.glob("data/obj/dataUpdated/train/*.jpg")

## Perform Normalization!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
### I have not done it........

print(images_list)

[]


In [ ]:
#Create train.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

**Be patient and wait for it to converge, now what is "converge" please Google in your waiting time and utilize your time, atleast for now. 🙂️**

Image Enhanced Trainingg

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_training
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     16       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  16 0.150 BF
   1 max                2x 2/ 2    416 x 416 x  16 ->  208 x 208 x  16 0.003 BF
   2 conv     32       3 x 3/ 1    208 x 208 x  16 ->  208 x 208 x  32 0.399 BF
   3 max                2x 2/ 2    208 x 208 x  32 ->  104 x 104 x  32 0.001 BF
   4 conv     64       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  64 0.399 BF
   5 max                2x 2/ 2    104 x 104 x  64 ->   52 x  52 x  64 0.001 BF
   6 conv    128       3 x 3/ 1     52 x  52 x  64 ->   52 x  52 x 128 0.399 BF
   7 max                2x 2/ 2     52 x  52 x 128 ->   26 x  26 x 128 0.000 BF
  

In [ ]:
!cp cfg/yolov3_training.cfg cfg/yolov3_testing.cfg

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re
import cv2

classes = {"0": "bicycle", "1" : "boat", "2" : "bottle" , "3" : "bus", "4":"car", "5":"cat" ,"6" : "chair" , "7":"cup","8":"dog","9":"motorbike","10":"people","11":"table"}
key_list = list(classes.keys())
val_list = list(classes.values())

txt_file_paths = glob.glob("data/obj/dataUpdated/test/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()
        # print(file_path)
        if '% bbGt version=3\n' in lines:
          lines.remove('% bbGt version=3\n')
        text_converted = []
        for line in lines:
            j = line.split(' ')
            p = '.'.join(file_path.split('.')[:2])
            # print(file_path.split('.'))
            if os.path.exists(p):
              img = cv2.imread(p)
              if img is not None:
                size = img.shape
              x1, x2, y1, y2 = int(j[1]), int(j[2]), int(j[3]) + int(j[1]), int(j[4]) + int(j[2])
              xmax, xmin = max(x1,x2) , min(x1,x2)
              ymax, ymin = max(y1,y2), min(y1,y2)
              dw = 1./size[1]
              dh = 1./size[0]
              x = (xmin + xmax)/2.0
              y = (ymin + ymax)/2.0
              w = xmax - xmin
              h = ymax - ymin
              x = x*dw
              w = w*dw
              y = y*dh
              h = h*dh
              text = "{} {} {} {} {}".format(key_list[val_list.index(j[0].lower())], x,y,w,h)
              text_converted.append(text)
        # Write file
        os.remove(file_path)
        filename = file_path.split('/')[4].split('.')[0] + '.txt'
        file_path = '/'.join(file_path.split('/')[:4]) + '/' + filename
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

In [ ]:
#Create train.txt file
images_list = glob.glob("data/obj/dataUpdated/test/*.jpg")
file = open("data/test.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

In [ ]:
!./darknet detector map data/obj.data cfg/yolov3_testing.cfg /content/gdrive/MyDrive/yolov3_w1/yolov3_training_last.weights -dont_show 

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     16       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  16 0.150 BF
   1 max                2x 2/ 2    416 x 416 x  16 ->  208 x 208 x  16 0.003 BF
   2 conv     32       3 x 3/ 1    208 x 208 x  16 ->  208 x 208 x  32 0.399 BF
   3 max                2x 2/ 2    208 x 208 x  32 ->  104 x 104 x  32 0.001 BF
   4 conv     64       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  64 0.399 BF
   5 max                2x 2/ 2    104 x 104 x  64 ->   52 x  52 x  64 0.001 BF
   6 conv    128       3 x 3/ 1     52 x  52 x  64 ->   52 x  52 x 128 0.399 BF
   7 max                2x 2/ 2     52 x  52 x 128 ->   26 x  26 x 128 0.000 BF
   8 conv    256  